## Problem 1
Using the code from the Basic Python Roundup lecture notebook, create three functions:

1. `scrape_book_results_page(page_num, headers)`: This function takes a page number and a headers dictionary as arguments and returns a dictionary with the following keys:
    - `page_url`: The URL of the books results page
    - `response`: The Response object of that page
    - `soup`: The BeautifulSoup object created from the source code
    - `book_urls`: A list of the URLs for each book on this page


2. `scrape_book_product_page(book_product_url, headers)`: This function takes a book product URL (the URL for the book product page) and a headers dictionary as arguments and returns a dictionary with the following keys:
    - `book_url`: The URL of the book product page 
    - `response`: The Response object of that page
    - `soup`: The BeautifulSoup object created from the source code


3. `scrape_book_range(page_range, filename, headers)`: This function takes a page range (`range` object), a filename for the a CSV file, and a headers dictionary as arguments and will use the other two functions to scrape the book information for every book found in the specified page range. This book information should be saved as separate rows in a CSV file (see if you can include the CSV file writing code in this function).

Make sure to include proper documentation (docstring) for your code.

**Before writing to CSV**, make the following changes to the book data:

1. Convert `price_in_pounds` value to `float` type.
2. Convert `avg_rating` to `int` type.
3. Extract the number of available books from the `num_books_available` string and convert to `int` type.

In [ ]:
import requests
from bs4 import BeautifulSoup as soup
import csv
import re

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}

def scrape_book_results_page(page_num, headers=headers):
    """scrape_book_results_page docstring"""
    pass

def scrape_book_product_page(book_url, headers=headers):
    """scrape_book_product_page docstring"""
    pass

def scrape_book_range(page_range, filename, headers=headers):
    """scrape_book_range docstring"""
    pass

## Problem 2
Take the code you've written above and create a module called `books2scrape`. This is just a file called `books2scrape` with a `py` extension. Make sure the module is located in your homework notebook working directory. Once you've created this module, import it and try to run the final function again.

What is the benefit of moving this code to a module? How does the functionality of a module compare to a class?

In [ ]:
# Your code here

### Written Answer